In [1]:
import tensorflow as tf
import os
import keras 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from keras.regularizers import l2

from keras.layers import Input, GlobalAveragePooling2D
from keras import models
from keras.models import Model
import numpy as np
from tensorflow_model_optimization.quantization.keras import vitis_quantize
from keras_flops import get_flops


shape_x = 224
shape_y = 224

X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
Y_train=np.random.randint(0,10,size=[60,])
X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
Y_test=np.random.randint(0,10,size=[60,])
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

nRows,nCols,nDims = X_train.shape[1:]
input_shape = (nRows, nCols, nDims)
classes = np.unique(Y_train)
nClasses = len(classes)
print(input_shape)
print(classes)

input_img = Input(shape=(shape_x, shape_y, 1))

### 1st layer
layer_1 = Conv2D(10, (1,1), padding='same', activation='relu')(input_img)
layer_1 = Conv2D(10, (3,3), padding='same', activation='relu')(layer_1)
pool1 = MaxPooling2D(pool_size=4)(layer_1)
layer_2 = Conv2D(10, (1,1), padding='same', activation='relu')(input_img)
layer_2 = Conv2D(10, (5,5), padding='same', activation='relu')(layer_2)
pool2 = MaxPooling2D(pool_size=4)(layer_2)
layer_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
layer_3 = Conv2D(10, (1,1), padding='same', activation='relu')(layer_3)
pool3 = MaxPooling2D(pool_size=4)(layer_3)
#mid_1 = tf.keras.layers.concatenate([layer_1, layer_2, layer_3], axis = 3)
mid_1 = tf.keras.layers.concatenate([pool1, pool2, pool3], axis = 3)
#mid_1 = tf.keras.layers.concatenate([layer_1, layer_3], axis = 3)
flat_1 = Flatten()(mid_1)

dense_1 = Dense(1200, activation='relu')(flat_1)
dense_2 = Dense(600, activation='relu')(dense_1)
dense_3 = Dense(150, activation='relu')(dense_2)
output = Dense(nClasses, activation='softmax')(dense_3)

model = Model([input_img], output)
model.summary()
Total_params = round(model.count_params()/10 ** 6,2)
FLOPs = round(get_flops(model, batch_size=1)/10 ** 9,2)
print(Total_params)
print(FLOPs)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# history = model.fit(X_train, Y_train, epochs=1)
# model.get_weights()[0].dtype
# quantizer = vitis_quantize.VitisQuantizer(model)
# quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
# quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
# quantized_model.save('tf2_mnist_classifier_quantized.h5')
# !vai_c_tensorflow2 \
# --model ./tf2_mnist_classifier_quantized.h5 \
# --arch ./arch.json \
# --output_dir ./xmodel_inception \
# --net_name deploy
# os.rename("./xmodel_inception/deploy.xmodel","./xmodel_inception/inception_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))

2022-03-29 16:58:23.720884: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-03-29 16:58:23.720915: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


(224, 224, 1)
[0 1 2 3 4 5 6 7 8 9]


2022-03-29 16:58:24.846878: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-03-29 16:58:24.846894: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-29 16:58:24.846910: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-03-29 16:58:24.847039: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 10) 20          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 224, 224, 10) 20          input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 224, 224, 1)  0           input_1[0][0]                    
______________________________________________________________________________________________

2022-03-29 16:58:25.542477: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-03-29 16:58:25.542588: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-03-29 16:58:25.562931: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.008ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number